<a href="https://colab.research.google.com/github/JoaoVLima/ActualSimplePDFValidator/blob/main/TDE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUCPR
## **João Victor de Lima**
### TDE01 - Construção de Interpretadores
**Desenvolver um programa que analise documentos no
formato simplepdf**, uma versão textual simplificada inspirada na estrutura interna de arquivos pdf.
O programa deverá **processar documentos simplepdf**, **validar sua estrutura segundo as regras formais da linguagem que aqui será definida**, **gerar estatísticas** e **transformações do conteúdo**.

# Explicação

## Formato SimplePDF

O SimplePDF é combosto por:

1. **Cabeçalho**: Todo documento começa com uma linha de cabeçalho
```
%SPDF-1.0
```

2. **Objetos**: O documento consiste em objetos numerados no formato
```
obj_id obj_gen obj
<< conteudo >>
endobj
```
No qual:
  - **obj_id** é um número inteiro único que identifica o objeto
  - **obj_gen** é um número de geração
(normalmente 0)

3. **Tipos de dados básicos**:
  - **Números**: inteiros (123) ou decimais (123.45)
  - **Strings**: delimitadas por parênteses: (Este é um texto)
  - **Nomes**: começam com /: /Nome
  - **Booleanos**: true ou false
  - **Null**: null
  - **Referências**: indicam referência a outro objeto: obj_id obj_gen R

4. **Estruturas de dados**:
  - **Dicionários**: pares chave/valor no formato: /Chave valor
  - **Arrays**: sequências de valores delimitadas por [ e ]
  - **Streams**: Blocos de dados entre stream e endstream

5. **Estrutura hierárquica**:
  - **Objeto catálogo (root)**: Define a estrutura do documento
  - **Objeto de páginas**: Mantém lista de páginas
  - **Objetos de página**: Contêm conteúdo e propriedades de cada página
  - **Objetos de conteúdo**: Contêm texto e elementos gráficos


6. **Comentários**:
  Linhas que começam com % (exceto o cabeçalho) são comentários

7. **Cross-reference table (xref)**: Tabela que lista a posição de cada objeto no arquivo:
```
xref
0 3
0000000000 65535 f
0000000010 00000 n
0000000079 00000 n
```

8. **Trailer**: Informações finais do documento:
```
trailer
<< /Size 3 /Root 1 0 R >>
startxref
183
%%EOF
```

## Funcionalidades Requeridas

O programa deve implementar as seguintes funcionalidades:

1. **Validação Estrutural**:
  - Verificar se o cabeçalho SimplePDF está correto
  - Validar a sintaxe de cada objeto e suas propriedades
  - Verificar se todas as referências apontam para objetos existentes
  - Validar a estrutura hierárquica (catálogo → páginas → página)
  - Verificar consistência da tabela xref
  - Validar o trailer

2. **Extração de Informações**:  
  - Conteúdo textual de cada página
  - Metadados do documento (título, autor, data)
  - Estrutura de páginas (quantidade, tamanho)
  - Fontes utilizadas
  - Estatísticas de objetos por tipo

3. **Transformações**:
  - Extração de todo texto para formato TXT;
  - Geração de sumário baseado na estrutura
  - Listagem hierárquica de objetos e suas dependências
  - Conversão de coordenadas de texto para formato mais legível

4. **Análise Avançada**:
  - Detecção de objetos não referenciados
  - Identificação de ciclos de referência
  - Análise de eficiência de armazenamento
  - Sugestões de otimização da estrutura

## Saída do Programa

O programa deve gerar um relatório com as seguintes seções:
```
VALIDAÇÃO:
[OK ou ERRO] Estrutura geral
[OK ou ERRO] Sintaxe de objetos
[OK ou ERRO] Referências
[OK ou ERRO] Tabela xref
[Detalhes de erros, se houver]

ESTATÍSTICAS:
Total de objetos: X
Objetos por tipo: Catalog=1, Pages=1, Page=Y, Font=Z, ...
Total de páginas: Y
Tamanho do documento: W bytes
Overhead estrutural: V bytes (P%)

CONTEÚDO:
Título: [título do documento]
Autor: [autor do documento]
Data de criação: [data]
Texto extraído: [primeiros 200 caracteres...]

ÁRVORE DE OBJETOS:
1: Catalog
  +- 2: Pages
    +- 3: Page
    +- 4: Font
    +- 5: Contents (stream)
6: Metadata

ANÁLISE AVANÇADA:
[Resultados de análises específicas definidas no arquivo de configuração]
```

# Código